# Práctico 4

## Transform Pattern

**"Transform pattern"** en el contexto de Machine Learning se refiere a la técnica de manipular y cambiar los datos de entrada antes de que sean utilizados por un modelo. 

Estos cambios pueden ayudar a mejorar el rendimiento del modelo y a hacer que el modelo sea más robusto ante variaciones en los datos de entrada.

Las transformaciones se aplican típicamente durante la etapa de pre_procesamiento de los datos y pueden implicar muchas técnicas diferentes, dependiendo del tipo de datos y del problema que se está tratando de resolver.

In [7]:
!pip install tensorflow==2.16.1

In [8]:
import tensorflow as tf
print(tf.__version__)


2.16.1


In [ ]:
%%time 

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Resizing, Rescaling
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt

# Cargar y normalizar el conjunto de datos 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convertir las etiquetas en one-hot
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

data_preprocessing = Sequential([
    Resizing(32, 32),  # Redimensionar a 32x32
    Rescaling(1./255)  # Normalización adicional después del redimensionamiento
])

# Define tu modelo
model = tf.keras.models.Sequential([
    data_preprocessing,

    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),  # Ajustar a 3 canales de entrada
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compila y entrena el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=256, epochs=5, validation_data=(x_test, y_test))

# Guarda el modelo
model.save('transform_pattern_conv.h5')


En este caso, las transformaciones de datos **(Resizing y Rescaling)** se definen como capas Keras y se agregan al inicio de tu modelo. 

Esto significa que estas transformaciones se aplicarán automáticamente a las imágenes a medida que pasen por el modelo, ya sea durante el entrenamiento o durante la inferencia. 

Además, como las capas de preprocesamiento son parte del modelo, se guardarán junto con el modelo cuando lo guardes con model.save(). 

## Experiment tracking
### wandb: https://wandb.ai/site


Primero, vamos a agregar experiment tracking utilizando wandb (Weights & Biases). Esto nos va a permitir monitorear los experimentos en tiempo real, guardar nuestros modelos , resultados, y podremos compartir experimentos con otros.

[Wandb collab full explained notebook ](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/intro/Intro_to_Weights_%26_Biases.ipynb#scrollTo=jufPgkgqz2eF)

### 👟 Run an experiment 

1.  **Start a new run** and pass in hyperparameters to track

2.  **Log metrics** from training or evaluation

3.  **Visualize results** in the dashboard

4. **Generate alerts** in the dashboard 

In [9]:
!pip install wandb==0.17.0

In [10]:
import wandb

print("wandb version:", wandb.__version__)


wandb version: 0.17.0


In [20]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import wandb
from wandb.integration.keras import WandbCallback

# Configura tu clave API directamente
wandb.login(key='213af0926e96a8bd86d4d352c1fa0e6d933474db')

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [13]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Resizing, Rescaling
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback
import numpy as np
import matplotlib.pyplot as plt
import random


# Definir un callback personalizado para wandb
class CustomWandbCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        wandb.log(logs, step=epoch)

# Launch 2 experiments, trying different dropout rates
for run in range(2):
    
    # Start a run, tracking hyperparameters
    wandb.init(
        project="ml-en-produccion",
        config={
            "activation_1": "relu",
            "dropout": random.uniform(0.01, 0.80),
            "optimizer": "adam",
            "loss": "categorical_crossentropy",
            "metric": "accuracy",
            "epoch": 3,
            "batch_size": 512,
        },
    )
    config = wandb.config
    
    # Cargar y normalizar el conjunto de datos 
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # Convertir las etiquetas en one-hot
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)

    # Define el preprocesamiento de la imagen
    data_preprocessing = Sequential([
        Resizing(32, 32),  # Redimensionar a 32x32
        Rescaling(1./255)  # Normalización adicional después del redimensionamiento
    ])

    # Define tu modelo
    model = tf.keras.models.Sequential([
        data_preprocessing,  

        tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(config.dropout),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(config.dropout),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    # Compila y entrena el modelo
    model.compile(optimizer=config.optimizer, loss=config.loss, metrics=[config.metric])

    # Add CustomWandbCallback to log metrics
    custom_wandb_callback = CustomWandbCallback()

    history = model.fit(x_train, y_train, batch_size=config.batch_size, epochs=config.epoch, 
                        validation_data=(x_test, y_test), callbacks=[custom_wandb_callback])
    
    wandb.finish()
    
    # Guardar el modelo
    model.save(f'model_run_{run}.h5')


Epoch 1/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - accuracy: 0.3149 - loss: 2.1526 - val_accuracy: 0.1016 - val_loss: 3.0394
Epoch 2/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.5183 - loss: 1.3476 - val_accuracy: 0.1004 - val_loss: 3.1182
Epoch 3/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 150s 1s/step - accuracy: 0.6045 - loss: 1.1221 - val_accuracy: 0.1572 - val_loss: 2.8141


accuracy,▁▆█
loss,█▃▁
val_accuracy,▁▁█
val_loss,▆█▁
accuracy,0.61496
loss,1.09503
val_accuracy,0.1572
val_loss,2.81415


Epoch 1/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 119s 1s/step - accuracy: 0.3622 - loss: 1.9514 - val_accuracy: 0.1040 - val_loss: 2.9159
Epoch 2/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.5648 - loss: 1.2294 - val_accuracy: 0.1032 - val_loss: 3.1708
Epoch 3/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.6380 - loss: 1.0186 - val_accuracy: 0.1324 - val_loss: 2.9724


accuracy,▁▆█
loss,█▃▁
val_accuracy,▁▁█
val_loss,▁█▃
accuracy,0.64656
loss,0.99442
val_accuracy,0.1324
val_loss,2.97235



##  W&B Alerts

**[W&B Alerts](https://docs.wandb.ai/guides/track/alert)** allows you to send alerts, triggered from your Python code, to your Slack or email. There are 2 steps to follow the first time you'd like to send a Slack or email alert, triggered from your code:

1) Turn on Alerts in your W&B [User Settings](https://wandb.ai/settings)

2) Add `wandb.alert()` to your code:

```python
wandb.alert(
    title="Low accuracy", 
    text=f"Accuracy is below the acceptable threshold"
)
```

In [14]:
import random 

# Start a wandb run
wandb.init(project="alerts-intro")

# Simulating a model training loop
acc_threshold = 0.3
for training_step in range(1000):

    # Generate a random number for accuracy
    accuracy = round(random.random() + random.random(), 3)
    print(f"Accuracy is: {accuracy}, {acc_threshold}")

    # 🐝 Log accuracy to wandb
    wandb.log({"Accuracy": accuracy})

    # 🔔 If the accuracy is below the threshold, fire a W&B Alert and stop the run
    if accuracy <= acc_threshold:
        # 🐝 Send the wandb Alert
        wandb.alert(
            title="Low Accuracy",
            text=f"Accuracy {accuracy} at step {training_step} is below the acceptable theshold, {acc_threshold}",
        )
        print("Alert triggered")
        break

# Mark the run as finished (useful in Jupyter notebooks)
wandb.finish()

Accuracy is: 1.122, 0.3
Accuracy is: 0.607, 0.3
Accuracy is: 0.588, 0.3
Accuracy is: 1.249, 0.3
Accuracy is: 1.182, 0.3
Accuracy is: 0.842, 0.3
Accuracy is: 0.991, 0.3
Accuracy is: 1.105, 0.3
Accuracy is: 1.336, 0.3
Accuracy is: 1.602, 0.3
Accuracy is: 1.5, 0.3
Accuracy is: 1.071, 0.3
Accuracy is: 0.928, 0.3
Accuracy is: 1.786, 0.3
Accuracy is: 0.751, 0.3
Accuracy is: 0.912, 0.3
Accuracy is: 0.959, 0.3
Accuracy is: 0.251, 0.3
Alert triggered


Accuracy,▅▃▃▆▅▄▄▅▆▇▇▅▄█▃▄▄▁
Accuracy,0.251


## H Tuning - wandb

In [29]:
import os
import wandb
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

# Cargar y normalizar el conjunto de datos
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# Usar solo una muestra del conjunto de datos de prueba
x_test_small = x_test[:1]
y_test_small = to_categorical(y_test[:1], 10)

# Definir el callback personalizado para wandb
class CustomWandbCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        wandb.log(logs, step=epoch)

# Definir la configuración del sweep
sweep_config = {
    'method': 'grid',  # el método de búsqueda de hiperparámetros
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'  
    },
    'parameters': {
        'learning_rate': {
            'values': [0.01]  # Un solo valor para minimizar el tiempo de ejecución
        },
        'batch_size': {
            'values': [64]  # Un solo valor para minimizar el tiempo de ejecución
        },
        'run_index': {
            'values': [0, 1]  # Índices de los modelos a cargar
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="Htuning")

# Definir la función de entrenamiento
def train():
    run = wandb.init()
    config = run.config
    
    print("Configuración de la ejecución:", config)
    
    # Mostrar archivos en el directorio actual
    print("Archivos en el directorio actual:", os.listdir('.'))
    
    # Cargar el modelo basado en el índice del run
    run_index = config.run_index
    model_path = f"model_run_{run_index}.h5"
    
    if os.path.exists(model_path):
        print(f"Cargando el modelo desde {model_path}")
        model = load_model(model_path)
    else:
        raise FileNotFoundError(f"Archivo de modelo {model_path} no encontrado.")
    
    print(f"Evaluando el modelo {model_path}")
    
    # Evaluar el modelo en los datos de prueba reducidos
    loss, accuracy = model.evaluate(x_test_small, y_test_small, verbose=0)
    
    # Loguear los resultados en wandb
    wandb.log({"val_loss": loss, "val_accuracy": accuracy})
    print(f"Modelo {model_path} - Pérdida de validación: {loss}, Precisión de validación: {accuracy}")
    
    wandb.finish()
    print("Ejecución de Wandb finalizada")

# Ejecutar el agente
print("Iniciando el sweep...")
wandb.agent(sweep_id, function=train)
print("Sweep finalizado")


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: cxwhl0r9
Sweep URL: https://wandb.ai/ortmlprod/Htuning/sweeps/cxwhl0r9
Iniciando el sweep...


wandb: Ctrl + C detected. Stopping sweep.


Sweep finalizado
